# Lab 4: Extending Logistic Regression

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('data/2014-clean.csv', encoding = 'latin1')
df_adjust = df

# Encodes education into a much more manageable prediction attribute
df_adjust.Education[df_adjust.Education < 4] = 1
df_adjust.Education[(df_adjust.Education > 3) & (df_adjust.Education < 13)] = 2
df_adjust.Education[(df_adjust.Education > 12) & (df_adjust.Education < 17)] = 3
df_adjust.Education[df_adjust.Education > 16] = 4

#Binary encode, 8 is not supervisor, <8 are unordered types of supervisors
df_adjust.SupervisoryStatus[df_adjust.SupervisoryStatus < 8] = 1
df_adjust.SupervisoryStatus[df_adjust.SupervisoryStatus == 8] = 0

#ensure data is integer encoded
df_adjust = df_adjust[np.isfinite(df_adjust['Education'])]
df_adjust = df_adjust[np.isfinite(df_adjust['SupervisoryStatus'])]
df_adjust.Education = df_adjust.Education.astype(int)
df_adjust.SupervisoryStatus = df_adjust.SupervisoryStatus.astype(int)

# Encodes Length of Service
df_adjust.LOS[df_adjust.LOS == '< 1'] = 0
df_adjust.LOS[df_adjust.LOS == '1-2'] = 1
df_adjust.LOS[df_adjust.LOS == '3-4'] = 2
df_adjust.LOS[df_adjust.LOS == '5-9'] = 3
df_adjust.LOS[df_adjust.LOS == '10-14'] = 4
df_adjust.LOS[df_adjust.LOS == '15-19'] = 5
df_adjust.LOS[df_adjust.LOS == '20-24'] = 6
df_adjust.LOS[df_adjust.LOS == '25-29'] = 7
df_adjust.LOS[df_adjust.LOS == '30-34'] = 8
df_adjust.LOS[df_adjust.LOS == '35+'] = 9
df_adjust.LOS[df_adjust.LOS == 'UNSP'] = np.NaN

#convert tointeger
df_adjust.Pay = df_adjust.Pay.astype(int)
df_adjust.LOS = df_adjust.LOS.astype(int)

df_adjust = df_adjust.dropna()

print(df_adjust.Education.value_counts())
print("------------------")
print(df_adjust.SupervisoryStatus.value_counts())
print("------------------")
print("Total: ", len(df_adjust))

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

ValueError: cannot convert float NaN to integer

In [81]:
print(df_adjust.Education.value_counts())
print(len(df_adjust))

2    492988
3    336146
4    175867
1      9440
Name: Education, dtype: int64
1014441


In [82]:
#df_adjust[['PseudoID', 'Name', 'Date', 'Agency','Station','Age','Education','PayPlan','Grade', 'LOS', 'Occupation', 'Category','Pay', 'SupervisoryStatus','Appointment', 'Schedule', 'NSFTP', 'AgencyName']].head()
removed_cols = df_adjust[['Agency','Age','Education','PayPlan', 'LOS', 'Category','Pay', 'SupervisoryStatus', 'Schedule', 'NSFTP', 'AgencyName']]
print(removed_cols.head())
print(removed_cols.describe())
for x in removed_cols:
    print(x, " : ", len(removed_cols[x].unique()))

  Agency    Age  Education PayPlan LOS Category     Pay  SupervisoryStatus  \
0   DJ01  35-39          3      GS   3        P  149993                  0   
1   DJ01  35-39          3      GS   3        P  149993                  0   
2   DJ01  40-44          3      GS   4        P  145504                  0   
3   DJ01  30-34          3      GS   3        P  124995                  0   
4   DJ01  30-34          3      GS   3        P  145827                  0   

  Schedule  NSFTP                     AgencyName  
0        F      1  OFFICES, BOARDS AND DIVISIONS  
1        F      1  OFFICES, BOARDS AND DIVISIONS  
2        F      1  OFFICES, BOARDS AND DIVISIONS  
3        F      1  OFFICES, BOARDS AND DIVISIONS  
4        F      1  OFFICES, BOARDS AND DIVISIONS  
          Education           Pay  SupervisoryStatus         NSFTP
count  1.014441e+06  1.014441e+06       1.014441e+06  1.014441e+06
mean   2.668782e+00  8.042660e+04       1.446679e-01  1.123286e+00
std    7.660622e-01  4.0

In [83]:
#one hot encode necessary data
dummies = pd.get_dummies(data=removed_cols, columns=['Agency', 'Age', 'PayPlan', 'Category', 'SupervisoryStatus', 'Schedule', 'NSFTP', 'AgencyName'])

In [84]:
#splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(dummies.drop('Education',1), dummies.Education, test_size=0.20, random_state=42)
X_train.head()

,LOS,Pay,Agency_DJ01,Agency_DJ02,Agency_DJ03,Agency_DJ06,Agency_DJ07,Agency_DJ08,Agency_DJ09,Agency_DJ10,...,AgencyName_SURFACE TRANSPORTATION BOARD,AgencyName_TRANSPORTATION SECURITY ADMINISTRATION,AgencyName_U.S. COAST GUARD,AgencyName_U.S. FISH AND WILDLIFE SERVICE,AgencyName_U.S. MARSHALS SERVICE,AgencyName_U.S. SECRET SERVICE,AgencyName_U.S. TRUSTEE PROGRAM,AgencyName_VETERAN EMPLOYMENT SERVICES OFFICE,AgencyName_VETERANS BENEFITS ADMINISTRATION,AgencyName_VETERANS HEALTH ADMINISTRATION
485504,4,69276,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
358345,5,87731,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
891253,1,42823,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
386719,3,76446,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
593279,7,44745,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
import numpy as np
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
        
blr = BinaryLogisticRegressionBase(0.1)
print(blr)

Base Binary Logistic Regression Object, Not Trainable


In [7]:
# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

            
blr = BinaryLogisticRegression(0.1)
print(blr)

Untrained Binary Logistic Regression Object


In [20]:
from sklearn.datasets import load_iris
import plotly

In [75]:
X = df_adjust.Pay
y = df_adjust.Education # make problem binary

plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'labels': np.unique(y),
          'values': np.bincount(y),
            'type': 'pie'}
fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Binary Class Distribution',
                'autosize':False,
                'width':500,
                'height':300}

plotly.offline.iplot(fig)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [12]:
from sklearn.metrics import accuracy_score

yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Accuracy of:  0.98


In [14]:
%%time
# can we do better? Maybe more iterations?
params = dict(eta=0.1,
              iterations=500)

blr = BinaryLogisticRegression(**params)
blr.fit(X,y)
print(blr)
yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

Binary Logistic Regression Object with coefficients:
[[-0.80547425]
 [-1.57588842]
 [-1.41637355]
 [ 2.33633735]
 [ 2.0138144 ]]
Accuracy of:  0.98
CPU times: user 815 ms, sys: 48.4 ms, total: 864 ms
Wall time: 829 ms


In [15]:
%%time
# now lets do some vectorized coding
from scipy.special import expit

class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

# use same params as defined above
blr = VectorBinaryLogisticRegression(**params)
blr.fit(X,y)
print(blr.w_)
yhat = blr.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

[[-0.80547425]
 [-1.57588842]
 [-1.41637355]
 [ 2.33633735]
 [ 2.0138144 ]]
Accuracy of:  0.98
CPU times: user 19.2 ms, sys: 2.22 ms, total: 21.4 ms
Wall time: 19.7 ms
